# Plant Leaf Disease Classifier

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from keras.optimizers import SGD, RMSprop

In [9]:
import os
import pandas as pd
import numpy as np


from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import permutation_importance

In [10]:
root = 'leaf_classif'
print(os.path.join(root, 'train'))

leaf_classif\train


In [11]:
files = [file for file in os.listdir(root) if os.path.isfile(os.path.join(root, file))]
print(files)

[]


# We extracted the pictures of leaves and gave them their corresponding labels

## All of the leaf pictures have the dimensions 6000 x 4000 and 96 dpi for both horizontal and vertical resolution

In [12]:
import random

main_dir = root  # Replace with your main directory

def label_from_folder_name(folder_name):
    if 'healthy' in folder_name:
        return 'healthy'  # Class label for healthy images
    elif 'diseased' in folder_name:
        return 'diseased'  # Class label for unhealthy images
    else:
        return None  # No specific label found
    
def custom_flow_from_directory(directory, target_size, batch_size):
    filenames = []
    labels = []
    i = 0

    for folder in os.listdir(directory):
        # first 2 folders
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                 for file in files:
                     filenames.append(os.path.join(root, file))
                     labels.append(label_from_folder_name(root))

    filenames = np.array(filenames)
    labels = np.array(labels, dtype=str)  # Ensure labels are strings

    return ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        pd.DataFrame({"filename": filenames, "class": labels}),
        x_col="filename",
        y_col="class",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

# Use the custom function to load data
batch_size = 32
img_height, img_width = 150, 150

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
valid_dir = os.path.join(main_dir, 'valid')

train_generator = custom_flow_from_directory(train_dir, (img_height, img_width), batch_size)
test_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)
valid_generator = custom_flow_from_directory(valid_dir, (img_height, img_width), batch_size)


Found 4274 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.


# Originally, we tried to run the model on all of the leaves, but found that we do not have the necessary hardware resources to do this.
- We tried to use Google Colab, but the service began to limit our usage of GPU and we encountered difficulty in trying to fully run the program.

# So, we decided to run the model on one type of leaf - Alstonia Scholaris.

In [13]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# We used a Convolutional Neural Network (CNN) from the Keras library to build a sequential model.

# Convolutional layers
- The first convolutional layer (Conv2D) has 32 filters, each with a 3x3 kernel. It applies the Rectified Linear Unit (ReLU) activation function.
- The second convolutional layer has 64 filters with a 3x3 kernel and ReLU activation.
- The third convolutional layer has 128 filters with a 3x3 kernel and ReLU activation.
- After each convolutional layer, a max-pooling layer (MaxPooling2D) with a 2x2 pool size is applied. This reduces the spatial dimensions of the representation and captures the most important information.

# Dense layers

- The first fully connected layer (Dense) has 128 neurons with ReLU activation. It processes the flattened features from the previous layer.
- The second and final fully connected layer has 1 neuron with a sigmoid activation function. This neuron outputs the probability of belonging to the positive class in binary classification tasks.

# Model Compilation

- The model is compiled using the Adam optimizer ('adam'), a popular optimization algorithm.
- The loss function used for training is binary crossentropy ('binary_crossentropy'), suitable for binary classification problems.
- The metric to monitor during training is accuracy ('accuracy').


In [14]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
steps_per_epoch_test = test_generator.samples // batch_size

epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
134/134 [==============================] - 734s 5s/step - loss: 0.6559 - accuracy: 0.6015 - val_loss: 0.6830 - val_accuracy: 0.6273
Epoch 2/10
134/134 [==============================] - 696s 5s/step - loss: 0.5465 - accuracy: 0.7272 - val_loss: 0.7343 - val_accuracy: 0.6636
Epoch 3/10
134/134 [==============================] - 697s 5s/step - loss: 0.4536 - accuracy: 0.7883 - val_loss: 0.5778 - val_accuracy: 0.7273
Epoch 4/10
134/134 [==============================] - 1481s 11s/step - loss: 0.3437 - accuracy: 0.8545 - val_loss: 0.5284 - val_accuracy: 0.7545
Epoch 5/10
134/134 [==============================] - 1983s 15s/step - loss: 0.2820 - accuracy: 0.8842 - val_loss: 0.5131 - val_accuracy: 0.8182
Epoch 6/10
134/134 [==============================] - 1979s 15s/step - loss: 0.2355 - accuracy: 0.9073 - val_loss: 0.5761 - val_accuracy: 0.7636
Epoch 7/10
134/134 [==============================] - 975s 7s/step - loss: 0.2029 - accuracy: 0.9202 - val_loss: 0.4438 - val_accuracy: 

In [15]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

4/4 [==============================] - 20s 5s/step - loss: 0.5602 - accuracy: 0.8364


In [ ]:
# get features and labels from test generator
X_test, Y_test = zip(*[(x, y) for x, y in test_generator])

# flatten features
X_test_flat = [item for sublist in X_test for item in sublist]


# calc permutation importance
result = permutation_importance(model, X_test_flat, Y_test, n_repeats=10, random_state=40, n_jobs=-1)
feature_importance = result.importances_mean

# show feature importance
feature_names = [f"Feature {i}" for i in range(len(feature_importance))]

plt.barh(feature_names, feature_importance)
plt.xlabel('Permutation Importance')
plt.title('Feature Importance')
plt.show()

Fine tune CNN model - doubling number of epochs to 20, reducing steps size by half

In [16]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
steps_per_epoch_test = test_generator.samples // (batch_size * 2)

epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
67/67 [==============================] - 427s 6s/step - loss: 0.1131 - accuracy: 0.9576 - val_loss: 0.4490 - val_accuracy: 0.8281
Epoch 2/20
67/67 [==============================] - 413s 6s/step - loss: 0.0818 - accuracy: 0.9743 - val_loss: 0.4763 - val_accuracy: 0.8438
Epoch 3/20
67/67 [==============================] - 418s 6s/step - loss: 0.0911 - accuracy: 0.9655 - val_loss: 0.6284 - val_accuracy: 0.7969
Epoch 4/20
67/67 [==============================] - 417s 6s/step - loss: 0.0805 - accuracy: 0.9728 - val_loss: 0.6238 - val_accuracy: 0.8281
Epoch 5/20
67/67 [==============================] - 418s 6s/step - loss: 0.0833 - accuracy: 0.9664 - val_loss: 0.6591 - val_accuracy: 0.8594
Epoch 6/20
67/67 [==============================] - 414s 6s/step - loss: 0.0671 - accuracy: 0.9790 - val_loss: 0.8000 - val_accuracy: 0.8125
Epoch 7/20
67/67 [==============================] - 407s 6s/step - loss: 0.0583 - accuracy: 0.9784 - val_loss: 0.6259 - val_accuracy: 0.8125
Epoch 8/20
67

In [17]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

2/2 [==============================] - 12s 6s/step - loss: 0.8206 - accuracy: 0.7969


Use a model built from a pretrained model

In [18]:
# Use pretrained model
from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Define input image size expected by MobileNet
img_width, img_height = 224, 224

# Load the MobileNet model without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification using sigmoid activation

# Combine base model with custom top layers
mobileModel = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
mobileModel.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [19]:
# Train the model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
134/134 [==============================] - 844s 6s/step - loss: 0.0494 - accuracy: 0.9836 - val_loss: 0.8153 - val_accuracy: 0.8091
Epoch 2/10
134/134 [==============================] - 810s 6s/step - loss: 0.0254 - accuracy: 0.9913 - val_loss: 0.8773 - val_accuracy: 0.8455
Epoch 3/10
134/134 [==============================] - 814s 6s/step - loss: 0.0142 - accuracy: 0.9956 - val_loss: 0.8577 - val_accuracy: 0.8545
Epoch 4/10
134/134 [==============================] - 817s 6s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.9834 - val_accuracy: 0.8364
Epoch 5/10
134/134 [==============================] - 808s 6s/step - loss: 7.1019e-04 - accuracy: 1.0000 - val_loss: 1.0705 - val_accuracy: 0.8273
Epoch 6/10
134/134 [==============================] - 1235s 9s/step - loss: 4.6795e-04 - accuracy: 1.0000 - val_loss: 1.1161 - val_accuracy: 0.8364
Epoch 7/10
134/134 [==============================] - 2011s 15s/step - loss: 3.4555e-04 - accuracy: 1.0000 - val_loss: 1.1541 - val_a

In [20]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

4/4 [==============================] - 48s 11s/step - loss: 0.8397 - accuracy: 0.5273
Test Loss: 0.8397
Test Accuracy: 52.73%


In [21]:
# Fine tune the pretrained model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
67/67 [==============================] - 1035s 15s/step - loss: 1.6930e-04 - accuracy: 1.0000 - val_loss: 1.3276 - val_accuracy: 0.8125
Epoch 2/20
67/67 [==============================] - 1021s 15s/step - loss: 1.4438e-04 - accuracy: 1.0000 - val_loss: 0.9707 - val_accuracy: 0.8594
Epoch 3/20
67/67 [==============================] - 1025s 15s/step - loss: 1.4007e-04 - accuracy: 1.0000 - val_loss: 0.5036 - val_accuracy: 0.8906
Epoch 4/20
67/67 [==============================] - 1025s 15s/step - loss: 1.4237e-04 - accuracy: 1.0000 - val_loss: 1.0700 - val_accuracy: 0.8906
Epoch 5/20
67/67 [==============================] - 1001s 15s/step - loss: 1.2913e-04 - accuracy: 1.0000 - val_loss: 1.3622 - val_accuracy: 0.8281
Epoch 6/20
67/67 [==============================] - 414s 6s/step - loss: 1.1160e-04 - accuracy: 1.0000 - val_loss: 0.9927 - val_accuracy: 0.8281
Epoch 7/20
67/67 [==============================] - 440s 7s/step - loss: 1.1852e-04 - accuracy: 1.0000 - val_loss: 0.863

In [22]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

4/4 [==============================] - 47s 11s/step - loss: 0.8397 - accuracy: 0.5273
Test Loss: 0.8397
Test Accuracy: 52.73%


In [32]:
from keras.preprocessing import image

images_to_predict_dir = "predict"

def predict_images_in_folder(model, folder_path):
    predictions = []
    image_paths = []
    print(folder_path)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.JPG')):
                img_path = os.path.join(root, file)
                image_paths.append(img_path)
                prediction = predict_image(model, img_path)
                predictions.append(prediction)
    return image_paths, predictions

# Function to predict a single image
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    if prediction < 0.5:
        return "Healthy"
    else:
        return "Unhealthy"

image_paths, predictions = predict_images_in_folder(model, images_to_predict_dir)

predict


In [33]:
import matplotlib.pyplot as plt

num_images = len(image_paths)
num_cols = 5
num_rows = -(-num_images // num_cols)  # Ceiling division

plt.figure(figsize=(15, 3 * num_rows))
for i, (img_path, prediction) in enumerate(zip(image_paths, predictions)):
    plt.subplot(num_rows, num_cols, i + 1)
    img = image.load_img(img_path, target_size=(img_height, img_width))
    plt.imshow(img)
    plt.title(f"{os.path.basename(img_path)}\nPrediction: {prediction}")
    plt.axis('off')

plt.tight_layout()
plt.show()

<Figure size 1500x0 with 0 Axes>